In [2]:
#######用我跑!!!!

# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 19:09:26 2017

@author: user
"""


import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv
from pandas import Series, DataFrame
import pandas as pd
print("-----STAR-----")
print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
print("sccessful link")

cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[X_loser_candinates_20161020001]")
results = cursor.fetchall()
#print(results)

cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[O_winner_candinates_20161020001]")
results_1 = cursor.fetchall()
#print(results_1)


count=0 
for i in results:
    count+=1
    #print ('%d:%s' % (count, i))
    
count_1 = 0
for i in results_1:
    count_1+=1
print("getting data sccessful")

def getDocDistance(a, b):
    if LA.norm(a)==0 or LA.norm(b)==0:
        return -1  
    return round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 5) 
def getDocSimilarity(wordFrequencyPair, minTimes=1):
    #v1 = var1
    #v2 = var2
    result = 0
    try:
        result = getDocDistance(v1, v2)
    except(RuntimeError, TypeError, NameError):
        pass        
    return result

with open('without_dict_o_x.csv', 'w', newline='') as f: 
    spamwriter = csv.writer(f, dialect='excel')
    headers = ['i', 'j', 'Similarity','i text','j text']
    spamwriter.writerow(headers)
    print("the csv document has created ")
    print("processing...........")
    for i in range(0 , len(results)):
        for j in range(i+1 ,len(results_1)):      
            for word1 in results[i]:                      
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word1 = word1.replace(chin, ' ')
                result= jieba.cut(word1) 
                result1 = " ".join(result)
                file1 = nltk.word_tokenize(result1)
                trainTokenFrequency = nltk.FreqDist(file1)              
            for word2 in results_1[j]:             
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word2 = word2.replace(chin, ' ')
                result_1= jieba.cut(word2) 
                result_1_after = " ".join(result_1)
                #print(result_1_after)
                result_1_after_token = nltk.word_tokenize(result_1_after)
                testTokenFrequency = nltk.FreqDist(result_1_after_token) 
            
            wordFrequencyPair = [trainTokenFrequency, testTokenFrequency]
            
            minTimes=1
            dict1 = {}
            for key in wordFrequencyPair[0].keys():
                if wordFrequencyPair[0].get(key, 0) >= minTimes:
                    dict1[key] = wordFrequencyPair[0].get(key, 0)
            dict2 = {}
            for key in wordFrequencyPair[1].keys():
                if wordFrequencyPair[1].get(key, 0) >= minTimes:
                    dict2[key] = wordFrequencyPair[1].get(key, 0)
            for key in dict2.keys():
                if dict1.get(key, 0) == 0:   #第一個字典中如果沒有第二字典中的字就把沒有的字加進去並設為0
                    dict1[key] = 0
            for key in dict1.keys():
                if dict2.get(key, 0) == 0:
                    dict2[key] = 0
            v1 = []
            for w1 in sorted(dict1.keys()):
                v1.append(dict1.get(w1))
            v2 = []    
            for w2 in sorted(dict2.keys()):
                v2.append(dict2.get(w2))
                
            def getDocSimilarity(wordFrequencyPair, minTimes=1):
                result = 0
                try:                
                    result = getDocDistance(v1, v2)
                    #print("getDocDistance try")
                except(RuntimeError, TypeError, NameError):
                    pass
                return result    
            data = [( i , j , getDocSimilarity(wordFrequencyPair, 1),result1,result_1_after)]   
            spamwriter.writerows(data)       #寫入csv
print("-----FINISH-----")
f.close()

-----STAR-----
try to link with DataBase:cate_test......
sccessful link
getting data sccessful
the csv document has created 
processing...........
-----FINISH-----


In [2]:
#################不同資料庫比

import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv

#連接資料庫
print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
cursor_1 = cnxn.cursor()
print("sccessful link")


#印出前十筆資料(指定在dbo.mobile01_list資料表)

cursor.execute("select [BulletinPlatform] from dbo.PFP_20161020001")
results = cursor.fetchall()
print(results)

cursor.execute("select [BulletinPlatform] from dbo.NPP_20161020001")
results_1 = cursor.fetchall()
print(results_1)


count=0 
  

##來ㄌ!去標點!來!
for i in results:
    count+=1
    ##print ('%d: %s' % (count, i))
for times in range(0,count):
    chineseFilter1 = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
    for chin in chineseFilter1:
        results[times] = str(results[times])
        results[times] = results[times].replace(chin,'')
    results[times] = jieba.cut(results[times])
    results[times] = " ".join(results[times])

count_1 = 0

for i in results_1:
    count_1+=1
    ##print ('%d: %s' % (count, i))
for times in range(0,count_1):
    chineseFilter1 = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
    for chin in chineseFilter1:
        results_1[times] = str(results_1[times])
        results_1[times] = results_1[times].replace(chin,'')
    results_1[times] = jieba.cut(results_1[times])
    results_1[times] = " ".join(results_1[times])

print(results)
print(results_1)

try to link with DataBase:cate_test......
sccessful link
[('1.捍衛原住民族基本權益：消除一切形式族群歧視，承認原住民族固有權利，尊重原住民族特殊地位，建立原住民族與國家形成對等的伙伴關係。2.完成原住民族立法自治：還原歷史真相，尊重原住民族意願、集體權益及自主自決精神，原住民族當家做主。3.鞏固原住民族土地及自然主權：儘速完成「原住民族土地及海域法」，歸還原住民族傳統土地，其水資源應充分回歸自主；發展航海文化，保障海洋民族權益。4.建立原住民族照護及醫療體系：制定「原住民族健康法」，充實原鄉醫療資源，推動老人長期照護，保障原住民族整體健康及生命安全。5.推動原住民族部落產業：制定「原住民族經濟發展法」，國家在部落投資，提供資金協助，提高經濟誘因，促進原住民「在地就業及創業」，永續經營。6.確立原住民族教育主體性：中央設置原住民族教育專責機構，建構一貫性原住民族教育體制，機會均等，資源優先。課綱微調，應尊重部落歷史。7.尊重原住民殘障同胞基本人權：維護身心自主權、生存權、教育權及工作權，扶助自力，使其活得有尊嚴。8.建構都市原住民居住正義機制：在都會區劃編原住民特色的聚落環境，永續發展，脫離無殼鍋牛之困境，建立優質生活環境。9.疼惜天然災害重創災難之災民：要求政府落實「安置重建」工作，並在災區推動「產業重建計畫」，永續發展災區經濟與生計，不再對家破人亡的災民落井下石。10.維護原住民族傳統生物多樣性知識：制定「原住民族傳統生物多樣性知識保護法」，尊重保存並促進永續利用及創新原住民族傳統生物多樣性知識。', ), ('一、終結藍、綠：禁止國、民兩黨立法院裡相互勾結協商，圖利各大家族財團、以各種名目榨乾人民血汗錢。 &nbsp;<BR><BR>二、民生有保障：支持廣電三法增加必載頻道，降低有線電視台費率，減輕民眾負擔。 &nbsp;<BR><BR>三、農民有依靠：農為國之根本大計、政府收購稻穀價格應跟每年物價指數調漲幅度一致，保障農民生活水平得以提升。 &nbsp;<BR><BR>四、做工有代價：勞工退休金與政府官員的差距相差實在太大，朝容主張提升至3萬，才能讓勞工家庭一輩子的辛苦有代價。 五、活得有尊嚴：老人的照顧成了家庭的壓力，利用台糖土地設立樂活銀髮村讓一般老人家能聚在一起談天、生活，減輕年輕人的壓力，讓

In [18]:
#######用我跑!!!!

# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 19:09:26 2017

@author: user
"""


import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv
from pandas import Series, DataFrame
import pandas as pd
print("-----STAR-----")
print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
print("sccessful link")

cursor.execute("select [BulletinPlatform] from dbo.NP_20161020001")
results = cursor.fetchall()
#print(results)

cursor.execute("select [BulletinPlatform] from dbo.rest_party_20161020001")
results_1 = cursor.fetchall()
#print(results_1)


count=0 
for i in results:
    count+=1
    #print ('%d:%s' % (count, i))
    
count_1 = 0
for i in results_1:
    count_1+=1
print("getting data sccessful")

def getDocDistance(a, b):
    if LA.norm(a)==0 or LA.norm(b)==0:
        return -1  
    return round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 5) 
def getDocSimilarity(wordFrequencyPair, minTimes=1):
    #v1 = var1
    #v2 = var2
    result = 0
    try:
        result = getDocDistance(v1, v2)
    except(RuntimeError, TypeError, NameError):
        pass        
    return result

with open('21new_NPnRESTPARTY_20161020001.csv', 'w', newline='') as f: 
    spamwriter = csv.writer(f, dialect='excel')
    headers = ['SIX_i', 'COUNTY_j', 'Similarity','PFP_i text','NPP_j text']
    spamwriter.writerow(headers)
    print("the csv document has created ")
    print("processing...........")
    for i in range(0 , len(results)):
        for j in range(0 ,len(results_1)):      
            for word1 in results[i]:                      
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word1 = word1.replace(chin, ' ')
                result= jieba.cut(word1) 
                result1 = " ".join(result)
                file1 = nltk.word_tokenize(result1)
                trainTokenFrequency = nltk.FreqDist(file1)              
            for word2 in results_1[j]:             
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word2 = word2.replace(chin, ' ')
                result_1= jieba.cut(word2) 
                result_1_after = " ".join(result_1)
                ####print(result_1_after)
                result_1_after_token = nltk.word_tokenize(result_1_after)
                testTokenFrequency = nltk.FreqDist(result_1_after_token) 
            
            wordFrequencyPair = [trainTokenFrequency, testTokenFrequency]
            
            minTimes=1
            dict1 = {}
            for key in wordFrequencyPair[0].keys():
                if wordFrequencyPair[0].get(key, 0) >= minTimes:
                    dict1[key] = wordFrequencyPair[0].get(key, 0)
            dict2 = {}
            for key in wordFrequencyPair[1].keys():
                if wordFrequencyPair[1].get(key, 0) >= minTimes:
                    dict2[key] = wordFrequencyPair[1].get(key, 0)
            for key in dict2.keys():
                if dict1.get(key, 0) == 0:   #第一個字典中如果沒有第二字典中的字就把沒有的字加進去並設為0
                    dict1[key] = 0
            for key in dict1.keys():
                if dict2.get(key, 0) == 0:
                    dict2[key] = 0
            v1 = []
            for w1 in sorted(dict1.keys()):
                v1.append(dict1.get(w1))
            v2 = []    
            for w2 in sorted(dict2.keys()):
                v2.append(dict2.get(w2))
                
            def getDocSimilarity(wordFrequencyPair, minTimes=1):
                result = 0
                try:                
                    result = getDocDistance(v1, v2)
                    #print("getDocDistance try")
                except(RuntimeError, TypeError, NameError):
                    pass
                return result    
            data = [( i , j , getDocSimilarity(wordFrequencyPair, 1),result1,result_1_after)]   
            spamwriter.writerows(data)       #寫入csv
print("-----FINISH-----")
f.close()

-----STAR-----
try to link with DataBase:cate_test......
sccessful link
getting data sccessful
the csv document has created 
processing...........
-----FINISH-----
